<a href="https://colab.research.google.com/github/cow-coding/ML-DL-Study/blob/master/Python%EC%9D%84%20%EC%9D%B4%EC%9A%A9%ED%95%9C%20%EA%B0%9C%EC%9D%B8%ED%99%94%20%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C/RS_Chapter3_CF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# Data Load

In [ ]:
DATA = '/content/drive/MyDrive/dataset/rec/'
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(DATA+'u.user', sep='|', names=u_cols, encoding='latin-1')
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
          'Action', 'Adventure', 'Animation', 'Children\s', 'Comedy', 'Crime', 'Documentary',
          'Drama', 'Fantasy', 'File-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
          'Thriller', 'War', 'Western']
movies = pd.read_csv(DATA+'u.item', sep='|', names=i_cols, encoding='latin-1')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(DATA+'u.data', sep='\t', names=r_cols, encoding='latin-1')
# timestamp 제거 
ratings = ratings.drop('timestamp', axis=1)
# movie ID와 title 빼고 다른 데이터 제거
movies = movies[['movie_id', 'title']]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

## Base Code

In [ ]:
x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

In [ ]:
def RMSE(y_true, y_pred):
  return np.sqrt(np.mean((np.array(y_true)-np.array(y_pred))**2))

def score(model):
  id_pairs = zip(x_test['user_id'], x_test['movie_id'])
  y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
  y_true = np.array(x_test['rating'])

  return RMSE(y_true, y_pred)

In [ ]:
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

# Collaborative Filtering

## Basic CF

### cosine similarity

In [ ]:
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)

In [ ]:
rating_matrix

movie_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,1633,1634,1635,1636,1637,1638,1639,1640,1641,1642,1643,1644,1645,1646,1647,1648,1650,1651,1652,1655,1658,1659,1660,1661,1662,1663,1664,1665,1666,1669,1672,1673,1674,1675,1676,1677,1678,1679,1680,1682
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,4.0,3.0,NaN,NaN,4.0,1.0,5.0,NaN,2.0,5.0,5.0,5.0,5.0,5.0,3.0,NaN,NaN,NaN,NaN,4.0,NaN,3.0,4.0,3.0,2.0,4.0,NaN,3.0,3.0,5.0,NaN,NaN,1.0,2.0,2.0,3.0,4.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,3.0,NaN,NaN,4.0,3.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
user_similarity

user_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.112711,0.015340,0.054268,0.275918,0.294322,0.318933,0.233963,0.081010,0.247806,0.230273,0.242415,0.333347,0.207907,0.133529,0.307084,0.164850,0.374196,0.055722,0.203217,0.142195,0.273804,0.309271,0.240086,0.232888,0.191503,0.137169,0.230552,0.055607,0.133625,0.043563,0.184908,0.020390,0.038747,0.021573,0.000000,0.229570,0.157187,0.023114,0.020760,...,0.119567,0.085001,0.113964,0.166623,0.228619,0.046832,0.193849,0.232470,0.136650,0.308199,0.022910,0.017931,0.454600,0.084008,0.200525,0.265535,0.035204,0.216923,0.274060,0.127500,0.227348,0.042246,0.035527,0.239863,0.144390,0.192941,0.173299,0.104836,0.263192,0.403370,0.313963,0.076492,0.157148,0.127723,0.154186,0.073739,0.196038,0.118767,0.139101,0.300915
2,0.112711,1.000000,0.079500,0.074899,0.040123,0.169787,0.050112,0.060297,0.160912,0.090655,0.033194,0.063199,0.163082,0.182400,0.315303,0.106872,0.184206,0.140793,0.072116,0.029327,0.162202,0.044782,0.051479,0.176322,0.043111,0.279324,0.116660,0.019000,0.122307,0.198693,0.075611,0.201515,0.156556,0.214197,0.028668,0.029608,0.018536,0.063706,0.164634,0.142229,...,0.084741,0.137180,0.213804,0.153756,0.062721,0.069701,0.207827,0.000000,0.030265,0.122474,0.040592,0.185538,0.070829,0.271078,0.014873,0.269287,0.184455,0.113436,0.095618,0.171407,0.157322,0.029132,0.221016,0.088649,0.000000,0.074138,0.043563,0.399427,0.029227,0.062434,0.131249,0.130110,0.203002,0.228272,0.201086,0.146527,0.162226,0.114987,0.087461,0.077153
3,0.015340,0.079500,1.000000,0.253842,0.000000,0.043451,0.038200,0.078935,0.062461,0.044844,0.030494,0.038706,0.144111,0.029570,0.050784,0.018409,0.035255,0.000000,0.113572,0.026941,0.097312,0.018061,0.018738,0.037380,0.016974,0.093833,0.020968,0.059346,0.129213,0.040942,0.171917,0.078950,0.381683,0.116767,0.158015,0.125798,0.017028,0.054022,0.243795,0.253436,...,0.023355,0.174031,0.000000,0.110306,0.027500,0.049396,0.014440,0.012761,0.012512,0.059141,0.000000,0.290694,0.031548,0.028957,0.008539,0.195364,0.401528,0.096193,0.032553,0.117262,0.031469,0.117086,0.204324,0.043115,0.081880,0.013622,0.000000,0.070090,0.000000,0.012075,0.009582,0.000000,0.084322,0.013559,0.120914,0.016153,0.140329,0.049711,0.068189,0.000000
4,0.054268,0.074899,0.253842,1.000000,0.043033,0.038723,0.066386,0.140270,0.077534,0.032869,0.040376,0.076873,0.074388,0.023492,0.074713,0.036561,0.070020,0.031808,0.095161,0.064210,0.114755,0.073237,0.026583,0.131609,0.000000,0.096984,0.093701,0.116133,0.111579,0.088938,0.080188,0.137879,0.273561,0.202923,0.066690,0.033763,0.086663,0.060353,0.050438,0.181206,...,0.046384,0.104289,0.000000,0.073750,0.099483,0.109006,0.117104,0.014256,0.000000,0.068758,0.000000,0.027824,0.072448,0.043134,0.000000,0.148088,0.098770,0.054586,0.092361,0.108129,0.081599,0.116273,0.057426,0.092766,0.022586,0.018036,0.089418,0.122828,0.011706,0.037472,0.067798,0.049457,0.103674,0.188501,0.115071,0.000000,0.162039,0.123413,0.110235,0.061802
5,0.275918,0.040123,0.000000,0.043033,1.000000,0.162986,0.285281,0.233159,0.048179,0.152221,0.255492,0.085449,0.261107,0.154944,0.080046,0.174434,0.082520,0.227350,0.134050,0.192048,0.143168,0.353892,0.351733,0.119310,0.252822,0.126319,0.076222,0.259987,0.023909,0.112982,0.036027,0.188271,0.000000,0.003451,0.008406,0.000000,0.208344,0.198293,0.000000,0.003595,...,0.049695,0.031924,0.063997,0.082500,0.157787,0.036496,0.151067,0.234199,0.104273,0.190302,0.045626,0.000000,0.270096,0.081642,0.106300,0.139595,0.000000,0.163566,0.296038,0.122976,0.148807,0.080082,0.000000,0.213302,0.048397,0.193233,0.110437,0.041672,0.233586,0.283435,0.248495,0.055637,0.081169,0.087995,0.123285,0.020408,0.107635,0.114041,0.101231,0.20829

In [ ]:
def CF_simple(user_id, movie_id):
  if movie_id in rating_matrix:
    sim_scores = user_similarity[user_id].copy()
    movie_ratings = rating_matrix[movie_id].copy()
    non_rating_idx = movie_ratings[movie_ratings.isnull()].index
    movie_ratings = movie_ratings.dropna()
    sim_scores = sim_scores.drop(non_rating_idx)
    mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
  else:
    mean_rating = 3.0

  return mean_rating

In [ ]:
score(CF_simple)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  


nan

### Paractice 3-1

Pearson correlation-coefficient

In [ ]:
### practice 3-1
corr = matrix_dummy.T.corr()    # MUST!!! Use user data when calculate similarity

In [ ]:
def CF_corr(user_id, movie_id):
  if movie_id in rating_matrix:
    sim_scores = corr[user_id].copy()                             # similarity user with others
    movie_ratings = rating_matrix[movie_id].copy()                # movie's user ratings

    non_rating_idx = movie_ratings[movie_ratings.isnull()].index
    movie_ratings = movie_ratings.dropna()

    sim_scores = sim_scores.drop(non_rating_idx)
    mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
  else:
    mean_rating = 3.0
  
  return mean_rating

In [ ]:
score(CF_corr)

1.0351871465938638

## Neighbor CF

In [ ]:
def score(model, neighbor_size=0):
  id_pairs = zip(x_test['user_id'], x_test['movie_id'])
  y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])
  y_true = np.array(x_test['rating'])

  return RMSE(y_true, y_pred)

In [ ]:
def CF_knn(user_id, movie_id, neighbor_size=0):
  if movie_id in rating_matrix:
    sim_scores = user_similarity[user_id].copy()
    movie_ratings = rating_matrix[movie_id].copy()
    non_rating_idx = movie_ratings[movie_ratings.isnull()].index
    movie_ratings = movie_ratings.drop(non_rating_idx)
    sim_scores = sim_scores.drop(non_rating_idx)

    if neighbor_size == 0:
      mean_rating = np.dor(sim_scores, movie_ratings) / sim_scores.sum()
    else:
      if len(sim_scores) > 1:
        neighbor_size = min(neighbor_size, len(sim_scores))
        sim_scores = np.array(sim_scores)
        movie_ratings = np.array(movie_ratings)
        user_idx = np.argsort(sim_scores)
        sim_scores = sim_scores = sim_scores[user_idx][-neighbor_size:]
        movie_ratings = movie_ratings[user_idx][-neighbor_size:]
        mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
      else:
        mean_rating = 3.0
  else:
    mean_rating = 3.0
  
  return mean_rating

In [ ]:
score(CF_knn, neighbor_size=30)

1.0117003434525427

## Recommendatin by input user

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

rating_matrix = ratings.pivot_table(values='rating', index='user_id', columns='movie_id')
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)
user_similarity

user_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.166931,0.047460,0.064358,0.378475,0.430239,0.440367,0.319072,0.078138,0.376544,0.311124,0.314678,0.439743,0.332107,0.181473,0.403987,0.215819,0.473875,0.109084,0.241498,0.198413,0.381042,0.446288,0.285399,0.335553,0.252066,0.129579,0.302737,0.129084,0.181933,0.114717,0.209719,0.032820,0.029046,0.036318,0.026590,0.278971,0.195617,0.057517,0.083809,...,0.112985,0.122914,0.138209,0.234845,0.276285,0.116184,0.260551,0.285488,0.204004,0.437751,0.057587,0.047805,0.569066,0.140963,0.232308,0.338279,0.058859,0.276885,0.383251,0.158199,0.302857,0.079341,0.083955,0.257381,0.234315,0.280072,0.279170,0.146437,0.354255,0.490353,0.369527,0.119482,0.274876,0.189705,0.197326,0.118095,0.314072,0.148617,0.179508,0.398175
2,0.166931,1.000000,0.110591,0.178121,0.072979,0.245843,0.107328,0.103344,0.161048,0.159862,0.129257,0.121268,0.223661,0.226758,0.455361,0.125330,0.223667,0.179218,0.101374,0.060499,0.204445,0.064443,0.146500,0.178323,0.148638,0.445594,0.185077,0.102455,0.175115,0.230078,0.081875,0.216032,0.167204,0.212489,0.075602,0.107012,0.079821,0.058597,0.268489,0.251593,...,0.211427,0.224852,0.278382,0.269172,0.101681,0.153909,0.345035,0.025506,0.023080,0.170131,0.039270,0.260726,0.116597,0.284455,0.049835,0.376975,0.328677,0.167278,0.122291,0.231709,0.244595,0.048471,0.390525,0.113863,0.091148,0.114631,0.288602,0.512473,0.047879,0.102243,0.156986,0.307942,0.358789,0.424046,0.319889,0.228583,0.226790,0.161485,0.172268,0.105798
3,0.047460,0.110591,1.000000,0.344151,0.021245,0.072415,0.066137,0.083060,0.061040,0.065151,0.075912,0.067345,0.198196,0.042780,0.129912,0.058530,0.031320,0.033788,0.104290,0.050696,0.165823,0.034462,0.030955,0.071430,0.037031,0.163477,0.021322,0.068035,0.153260,0.108480,0.190257,0.125907,0.386958,0.140309,0.247316,0.132368,0.012527,0.062250,0.284856,0.385311,...,0.064352,0.220405,0.063105,0.136337,0.083157,0.071430,0.048256,0.009207,0.034366,0.088333,0.000000,0.358316,0.046124,0.044585,0.006246,0.264406,0.434379,0.094758,0.055179,0.164234,0.069298,0.184687,0.286220,0.052784,0.070027,0.036707,0.023085,0.176416,0.000000,0.036302,0.031875,0.042753,0.163829,0.069038,0.124245,0.026271,0.161890,0.101243,0.133416,0.026556
4,0.064358,0.178121,0.344151,1.000000,0.031804,0.068044,0.091230,0.188060,0.101284,0.060859,0.104182,0.075324,0.120306,0.070670,0.148397,0.084951,0.053895,0.035582,0.158678,0.086565,0.189713,0.082280,0.061922,0.175470,0.084112,0.151322,0.074295,0.151293,0.143465,0.144516,0.070563,0.132090,0.339211,0.152106,0.196643,0.067042,0.111555,0.065307,0.228181,0.238689,...,0.073434,0.171210,0.027923,0.115243,0.106580,0.081942,0.151247,0.029705,0.032256,0.107485,0.000000,0.136295,0.055061,0.043939,0.000000,0.182571,0.171236,0.100374,0.132542,0.094344,0.082147,0.168100,0.232185,0.127523,0.053364,0.048234,0.073335,0.138921,0.023274,0.071446,0.052107,0.036784,0.133115,0.193471,0.146058,0.030138,0.196858,0.152041,0.170086,0.058752
5,0.378475,0.072979,0.021245,0.031804,1.000000,0.237286,0.373600,0.248930,0.056847,0.201427,0.321710,0.130664,0.345217,0.244741,0.096349,0.287556,0.084729,0.289495,0.116843,0.240738,0.176494,0.455057,0.409268,0.137635,0.329765,0.152247,0.080846,0.322380,0.052199,0.164811,0.041286,0.165350,0.000000,0.002489,0.009077,0.000000,0.243336,0.254872,0.000000,0.005292,...,0.048666,0.022342,0.062368,0.145534,0.231190,0.026398,0.191689,0.350941,0.175405,0.334656,0.038711,0.000000,0.342110,0.074416,0.168863,0.185523,0.000000,0.258688,0.423096,0.159031,0.211030,0.063369,0.000000,0.289893,0.085181,0.233524,0.143641,0.068424,0.324288,0.373698,0.338794,0.080580,0.094924,0.079779,0.148607,0.071459,0.239955,0.139595,0.152497,0.31394

In [ ]:
def recom_movie(user_id, n_items, neighbors_size=30):
  user_movie = rating_matrix.loc[user_id].copy()

  for movie in rating_matrix:
    if pd.notnull(user_movie.loc[movie]):
      user_movie.loc[movie] = 0
    else:
      user_movie.loc[movie] = CF_knn(user_id, movie, neighbors_size)
  
  movie_sort = user_movie.sort_values(ascending=False)[:n_items]
  recom_movies = movies.loc[movie_sort.index]
  recommendations = recom_movies['title']

  return recommendations

In [ ]:
recom_movie(user_id=2, n_items=5, neighbors_size=30)

movie_id
1293                     Ayn Rand: A Sense of Life (1997)
1189                              That Old Feeling (1997)
1467                                     Cure, The (1995)
1500    Prisoner of the Mountains (Kavkazsky Plennik) ...
318                       Everyone Says I Love You (1996)
Name: title, dtype: object

## Select Best Neighbor size

In [ ]:
rating_matrix = x_train.pivot_table(values='rating', index='user_id', columns='movie_id')
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)

In [ ]:
for neighbor_size in [10, 20, 30, 40, 50, 60]:
  print(f'Neighbor size = %d : RMSE = %.4f' % (neighbor_size, score(CF_knn, neighbor_size)))

Neighbor size = 10 : RMSE = 1.0274
Neighbor size = 20 : RMSE = 1.0141
Neighbor size = 30 : RMSE = 1.0117
Neighbor size = 40 : RMSE = 1.0108
Neighbor size = 50 : RMSE = 1.0117
Neighbor size = 60 : RMSE = 1.0124


In [ ]:
best = 2.0
count = 0

for neighbor_size in range(30, 50):
  s = score(CF_knn, neighbor_size)
  print(f'Neighbor size = %d : RMSE = %.4f' % (neighbor_size, s))
  if best >= s:
    best = s
  elif best < s:
    count += 1

  if count > 2: break

Neighbor size = 30 : RMSE = 1.0117
Neighbor size = 31 : RMSE = 1.0115
Neighbor size = 32 : RMSE = 1.0112
Neighbor size = 33 : RMSE = 1.0111
Neighbor size = 34 : RMSE = 1.0111
Neighbor size = 35 : RMSE = 1.0111
Neighbor size = 36 : RMSE = 1.0110
Neighbor size = 37 : RMSE = 1.0109
Neighbor size = 38 : RMSE = 1.0111
Neighbor size = 39 : RMSE = 1.0110


## Considered User bias

In [ ]:
rating_mean = rating_matrix.mean(axis=1)
rating_bias = (rating_matrix.T - rating_mean).T

In [ ]:
def CF_knn_bias(user_id, movie_id, neighbor_size=0):
  if movie_id in rating_bias:
    sim_scores = user_similarity[user_id].copy()
    movie_ratings = rating_bias[movie_id].copy()
    none_rating_idx = movie_ratings[movie_ratings.isnull()].index
    movie_ratings = movie_ratings.drop(none_rating_idx)
    sim_scores = sim_scores.drop(none_rating_idx)

    if neighbor_size == 0:
      prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
      prediction = prediction + rating_mean[user_id]
    else:
      if len(sim_scores) > 1:
        neighbor_size = min(neighbor_size, len(sim_scores))
        sim_scores = np.array(sim_scores)
        movie_ratings = np.array(movie_ratings)
        user_idx = np.argsort(sim_scores)
        sim_scores = sim_scores[user_idx][-neighbor_size:]
        movie_ratings = movie_ratings[user_idx][-neighbor_size:]
        prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
        prediction = prediction + rating_mean[user_id]
      else:
        prediction = rating_mean[user_id]
  else:
    prediction = rating_mean[user_id]
  
  return prediction

In [ ]:
score(CF_knn_bias, 30)

0.947945056774684

### Practice 3-2

In [ ]:
rating_matrix = ratings.pivot_table(values='rating', index='user_id', columns='movie_id')
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)

In [ ]:
def recom_movie_bias(user_id, n_items, neighbor_size=30):
  user_movie = rating_matrix.loc[user_id]

  for movie in rating_matrix:
    if pd.notnull(user_movie.loc[movie]):
      user_movie.loc[movie] = 0
    else:
      user_movie.loc[movie] = CF_knn_bias(user_id, movie, neighbor_size)

  movie_sort = user_movie.sort_values(ascending=True)[:n_items]
  recom_movies = movies.loc[movie_sort.index]
  recommendations = recom_movies['title']

  return recommendations

In [ ]:
recom_movie_bias(user_id=2, n_items=5, neighbor_size=30)

movie_id
1                            GoldenEye (1995)
1128                 Chungking Express (1994)
1127    Heidi Fleiss: Hollywood Madam (1995) 
1126                  Truman Show, The (1998)
1125          Old Man and the Sea, The (1958)
Name: title, dtype: object

In [ ]:
rating_matrix = x_train.pivot_table(values='rating', index='user_id', columns='movie_id')
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)

In [ ]:
for neighbor_size in [10, 20, 30, 40, 50, 60]:
  print(f'Neighbor size = %d : RMSE = %.4f' % (neighbor_size, score(CF_knn_bias, neighbor_size)))

Neighbor size = 10 : RMSE = 0.9619
Neighbor size = 20 : RMSE = 0.9501
Neighbor size = 30 : RMSE = 0.9479
Neighbor size = 40 : RMSE = 0.9476
Neighbor size = 50 : RMSE = 0.9486
Neighbor size = 60 : RMSE = 0.9495


## Significance Weighting

In [ ]:
rating_binary1 = np.array((rating_matrix > 0).astype(float))
rating_binary2 = rating_binary1.T
counts = np.dot(rating_binary1, rating_binary2)
counts = pd.DataFrame(counts, index=rating_matrix.index, columns=rating_matrix.index).fillna(0)

In [ ]:
def CF_knn_bias_sig(user_id, movie_id, neighbor_size=0, adv=False):
  if movie_id in rating_bias:
    sim_scores = user_similarity[user_id].copy()
    movie_ratings = rating_bias[movie_id].copy()
    no_rating = movie_ratings.isnull()
    common_counts = counts[user_id]
    low_significance = common_counts < SIG_LEVEL
    none_rating_idx = movie_ratings[no_rating | low_significance].index
    movie_ratings = movie_ratings.drop(none_rating_idx)
    sim_scores = sim_scores.drop(none_rating_idx)

    if neighbor_size == 0:
      prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
      prediction = prediction + rating_mean[user_id]
    else:
      if len(sim_scores) > MIN_RATINGS:
        neighbor_size = min(neighbor_size, len(sim_scores))
        sim_scores = np.array(sim_scores)
        movie_ratings = np.array(movie_ratings)
        user_idx = np.argsort(sim_scores)
        sim_scores = sim_scores[user_idx][-neighbor_size:]
        movie_ratings = movie_ratings[user_idx][-neighbor_size:]
        prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
        prediction = prediction + rating_mean[user_id]
      else:
        prediction = rating_mean[user_id]
  else:
    prediction = rating_mean[user_id]

  if adv:
    if prediction < 1.0: prediction = 1.0
    elif prediction > 5.0: prediction = 5.0
  
  return prediction

In [ ]:
# modify the score based on advanced function
def score(model, neighbor_size=0, adv=False):
  id_pairs = zip(x_test['user_id'], x_test['movie_id'])
  y_pred = np.array([model(user, movie, neighbor_size, adv) for (user, movie) in id_pairs])
  y_true = np.array(x_test['rating'])

  return RMSE(y_true, y_pred)

In [ ]:
SIG_LEVEL = 3
MIN_RATINGS = 2

score(CF_knn_bias_sig, 30)

0.9473127552897089

### Practice 3-3

In [ ]:
SIG_LEVEL = 3
MIN_RATINGS = 2

score(CF_knn_bias_sig, 30, True)

0.9460677611564304

## User based & Item based

In [ ]:
def score(model):
  id_pairs = zip(x_test['user_id'], x_test['movie_id'])
  y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
  y_true = np.array(x_test['rating'])

  return RMSE(y_true, y_pred)

In [ ]:
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

rating_matrix_t = np.transpose(rating_matrix)
matrix_dummy = rating_matrix_t.copy().fillna(0)
item_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
item_similarity = pd.DataFrame(item_similarity, index=rating_matrix_t.index, columns=rating_matrix_t.index)

In [ ]:
def CF_IBCF(user_id, movie_id):
  VERY_SMALL = 1e-10                                          # protect divide by zero error

  if movie_id in item_similarity:
    sim_scores = item_similarity[movie_id]
    user_rating = rating_matrix_t[user_id]
    non_rating_idx = user_rating[user_rating.isnull()].index
    user_rating = user_rating.dropna()
    sim_scores = sim_scores.drop(non_rating_idx)

    sum_sim = sim_scores.sum()

    if sum_sim <= 0: sum_sim = VERY_SMALL

    mean_rating = np.dot(sim_scores, user_rating) / sum_sim
  else:
    mean_rating = 3.0
  
  return mean_rating

In [ ]:
score(CF_IBCF)

1.0179936956971813